<a href="https://colab.research.google.com/github/RPGraciotti/BootCampAlura/blob/main/Projeto_final/TPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tpot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate

from tpot import TPOTClassifier
from tpot.builtins import StackingEstimator

from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report



In [ ]:
path = "https://raw.githubusercontent.com/RPGraciotti/BootCampAlura/main/Data/data_clean_ohe.csv"

In [ ]:
df_clean = pd.read_csv(path)
df_clean

In [ ]:
df_clean = df_clean.sample(frac = 1, random_state = 78329).reset_index(drop = True)
x_columns = df_clean.columns
y = df_clean.loc[:,"ICU"]
x = df_clean.drop(["PATIENT_VISIT_IDENTIFIER", "ICU", "WINDOW"], axis = 1)

In [ ]:
y = y.rename("target")

In [ ]:
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10, random_state = 78329)

TPOT

ACC

In [ ]:
ACC = TPOTClassifier(generations = 5, population_size = 20, cv = cv, 
                      scoring = "accuracy", verbosity = 2, random_state = 78329)

In [ ]:
ACC.fit(x, y)

In [ ]:
ACC.export("shuffle_random_state_acc.py")

PRECISION

In [ ]:
Prec = TPOTClassifier(generations = 5, population_size = 20, cv = cv, 
                      scoring = "precision", verbosity = 2, random_state = 78329)

In [ ]:
Prec.fit(x, y)

In [ ]:
Prec.export("shuffle_random_state_prec.py")

ROC AUC

In [ ]:
ROC = TPOTClassifier(generations = 5, population_size = 20, cv = cv, 
                      scoring = "roc_auc", verbosity = 2, random_state = 78329)


In [ ]:
ROC.fit(x, y)

In [ ]:
ROC.export("shuffle_random_state_roc.py")

RECALL

In [ ]:
REC = TPOTClassifier(generations = 50, population_size = 20, 
                      cv = cv, scoring = "recall", verbosity = 2, random_state = 78329)

In [ ]:
REC.fit(x, y)

In [ ]:
REC.export("shuffle_random_state_recall_50.py")

F1

In [ ]:
F1 = TPOTClassifier(generations = 5, population_size = 20, cv = cv, 
                      scoring = "f1", verbosity = 2, random_state = 78329)

In [ ]:
F1.fit(x, y)

In [ ]:
F1.export("shuffle_random_state_f1.py")